In [5]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from  lightgbm import LGBMClassifier
from collections import Counter
from sklearn.metrics import precision_recall_curve, auc
import pickle
import yaml

pd.set_option('display.max_columns', None)

In [6]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [ ]:
predfes

array([0.95291321, 0.95211721, 0.97290711, ..., 0.95670578, 0.96947052,
       0.95349868])

In [ ]:
df_test = pd.read_parquet('data_features/validation_audio_diarization.parquet')
df2_test = pd.read_parquet('data_features/validation_audio_features.parquet')
df_test = pd.merge(df_test, df2_test, how='left', left_on=['filename'], right_on=['applicationid'])

app_data = pd.read_excel('data_features/validation_audio.xlsx', header=1)
df_test = pd.merge(df_test, app_data, how='left', left_on=['filename'], right_on=['Файлы'])
df_test = df_test.query("credit_rule.notna()")
df_test.fillna(0, inplace=True)

audio_model = pickle.load(open('audio_model.pkl', 'rb'))
audio_calibrator = pickle.load(open('audio_calib.pkl', 'rb'))

pred = audio_model.predict_proba(df_test[config['audio_features']])[:, 1]
pred = audio_calibrator.predict(pred)
print(f"PRC_AUC: {pr_auc(df_test['credit_rule'], pred)}")
print(f"GINI: {roc_auc_score(df_test['credit_rule'], pred) * 2 - 1}")
df_test['pd'] = pred
df_test[['APPLICATIONID', 'pd']].to_parquet('./validation_audio_pd.parquet')

/home/ubuntu/.local/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[0.95291321 0.95211721 0.97290711 ... 0.95670578 0.96947052 0.95349868].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.